In [ ]:
import os
import sys

# 환경 감지 및 API 키 로드
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ 구글 코랩 환경: API 키 로드 완료")
except ImportError:
    # VS Code / Codespaces 환경 (.env 파일 사용)
    from dotenv import load_dotenv
    load_dotenv()
    if not os.getenv("GOOGLE_API_KEY"):
        print("⚠️ 경고: .env 파일에서 GOOGLE_API_KEY를 찾을 수 없습니다.")
    else:
        print("✅ VS Code/Codespaces 환경: API 키 로드 완료")

# 필수 라이브러리 설치 (없을 경우 자동 설치 안내 메시지)
try:
    import langchain_google_genai
    import pymupdf
    import chromadb
except ImportError:
    print("⚠️ 라이브러리가 설치되지 않았습니다. 아래 명령어를 터미널에 실행하세요:")
    print("pip install langchain langchain-community langchain-google-genai pymupdf chromadb python-dotenv")

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def read_pdf_and_split_text_improved(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    [개선된 함수] PyMuPDFLoader를 사용하여 PDF를 읽고 더 큰 단위로 분할합니다.
    """
    print(f"📂 PDF 로드 중: {pdf_path} ...")
    
    # 1. Loader 변경: PyMuPDFLoader가 표/레이아웃 처리에 훨씬 강력함
    loader = PyMuPDFLoader(pdf_path) 
    pages = loader.load()
    
    # 2. Splitter 설정: chunk_size를 1000으로 늘려 문맥(Context) 보존
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    splits = text_splitter.split_documents(pages)
    print(f"✅ 총 {len(splits)}개의 청크로 분할되었습니다.")
    
    # (디버깅) 첫 번째 청크 내용 확인
    if splits:
        print(f"\n[샘플 청크 내용]:\n{splits[10].page_content[:200]}...\n")
        
    return splits

# 실행 예시 (파일 경로는 환경에 맞게 수정 필요)
pdf_path = r'C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap13\2025학년도 서울 학교운동부 운영 계획(매뉴얼)_공문 발송용.pdf'

if os.path.exists(pdf_path):
    docs = read_pdf_and_split_text_improved(pdf_path)
else:
    print(f"❌ 파일을 찾을 수 없습니다: {pdf_path}")

In [ ]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# 1. 임베딩 모델 설정
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# 2. 벡터 저장소 생성 (메모리 모드)
# 영구 저장을 원하면 persist_directory="./chroma_db" 옵션 추가
if 'docs' in locals() and docs:
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding
    )
    print("✅ 벡터 저장소 생성 완료")

    # 3. 검색기 설정 (k=10으로 설정하여 38페이지 내용이 밀려나지 않도록 함)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
else:
    print("⚠️ 문서가 로드되지 않아 벡터 저장소를 생성할 수 없습니다.")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. LLM 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

# 2. 프롬프트 템플릿
rag_prompt_template = """
당신은 학교 운동부 운영 전문가입니다. 아래의 [Context]를 바탕으로 [Question]에 대해 상세히 답변해 주세요.
만약 [Context]에 없는 내용이라면 "제공된 문서에서 정보를 찾을 수 없습니다."라고 답하세요.

[Context]:
{context}

[Question]:
{question}
"""

rag_prompt = PromptTemplate.from_template(rag_prompt_template)

# 3. 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 4. RAG 체인 구성
if 'retriever' in locals():
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | rag_prompt
        | llm
        | StrOutputParser()
    )

    # 5. 실행 및 결과 출력
    query = "운동부 전지훈련 계획에 대해 알려줘"
    print(f"❓ 질문: {query}\n")
    response = rag_chain.invoke(query)
    print(f"💡 답변:\n{response}")
else:
    print("⚠️ 검색기가 설정되지 않았습니다.")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # 원본 모델 유지
    api_key=GOOGLE_API_KEY,
    temperature=0.3
)
chat.invoke('운동부 전지훈련 계획 작성방법 알려줘')
